In [ ]:
using Glob
using JSON
using LargeScaleAnalysis
using ProgressMeter
using PyPlot
using Statistics
using StatsBase

In [ ]:
include("../../ParsimoniousMonitoring/notebooks/thesis.jl")

### Analysis of self measurements

In [ ]:
files = glob("*.model.json", "../data/ping_v4_1580511600_1581116400_self_pairs/");

In [ ]:
models = map(x -> parsefile(DataSegmentationModel, x), files);

In [ ]:
nstates = map(x -> size(x.model, 1), models);

In [ ]:
nstatesdist = counts(nstates, maximum(nstates))
bar(1:length(nstatesdist), nstatesdist);

In [ ]:
# TODO: Check in traceroute that there is no "spurious" hops

In [ ]:
for m in models[nstates .>= 4]
    figure(figsize = (12,2))
    plot(coalesce.(m.data, NaN))
end

### Analysis of non-self measurements

In [ ]:
# TODO: Re-do analysis with 10% of the pairs instead of 1%.
# TODO: ACF insides the states?

In [ ]:
# TODO: Number of states vs. number of hops ?
# TODO: Comparer modeles appris sur 1 et 3 jours vs. les sous-sequences
# de 1 et 3 jours sur un modele appris sur 7 jours (est-ce que ca match ?).

In [ ]:
files = glob("*ndjson.model.json", "../data/ping_v4_1580511600_1581116400_noself_pairs/");
files_1d = glob("*ndjson_360.model.json", "../data/ping_v4_1580511600_1581116400_noself_pairs/");
files_3d = glob("*ndjson_1080.model.json", "../data/ping_v4_1580511600_1581116400_noself_pairs/");
files_14d = glob("*ndjson.model.json", "../data/ping_v4_1580511600_1581721200_noself_pairs/");

In [ ]:
models = map(x -> parsefile(DataSegmentationModel, x), files);
models_1d = map(x -> parsefile(DataSegmentationModel, x), files_1d);
models_3d = map(x -> parsefile(DataSegmentationModel, x), files_3d);
models_14d = map(x -> parsefile(DataSegmentationModel, x), files_14d);

In [ ]:
nstates = map(x -> size(x.model, 1), models);
nstates_1d = map(x -> size(x.model, 1), models_1d);
nstates_3d = map(x -> size(x.model, 1), models_3d);
nstates_14d = map(x -> size(x.model, 1), models_14d);

In [ ]:
fig, ax = subplots()
ax.boxplot([nstates_1d, nstates_3d, nstates, nstates_14d], labels = ["1 jour", "3 jours", "7 jours", "14 jours"], showfliers = false)
ax.set(ylabel = "Nombre d'états", ylim = (0, 17))
ax.grid(axis = "x")
save_thesis("atlas_nstates_dist_boxplot", hwr = 0.5, extra_axis_params = ["xtick={1,2,3,4}", "xticklabels={1 jour, 3 jours, 7 jours, 14 jours}"])

In [ ]:
# Verifier si pour une series donnee le nombre d'etats augmente toujours avec le temps.
# Chercher les series pour lesquelles le nombre d'etats entre 7 et 14 jours change peu/change beaucoup.

#### Relation CV / durée

In [ ]:
files = glob("*ndjson.model.json", "../data/ping_v4_1580511600_1581116400_01_pairs/");
models = map(x -> parsefile(DataSegmentationModel, x), files);

In [ ]:
# Compute avg. durations by states
durations, stds = [], []
# for model in models_14d
for model in models
    for (state, segments) in group(segments(model.state))
        data = model.data[segments]
        push!(durations, mean(length, segments))
        push!(stds, std(skipmissing(data)) / mean(skipmissing(data)))
    end
end

durations = durations[isfinite.(stds)]
stds = stds[isfinite.(stds)];

In [ ]:
maximum(stds)

In [ ]:
# Δ = 500
# groups = []
# for i in 1:Δ:5000
#     indices = (durations .>= i) .& (durations .< i+Δ)
#     push!(groups, stds[indices])
# end;

In [ ]:
Δ = 0.2
I = []
groups = []
for i in 0:Δ:2
    indices = (stds .>= i) .& (stds .< i+Δ)
    if sum(indices) <= 10
        break
    end
    push!(I, i + Δ)
    push!(groups, durations[indices])
end;

In [ ]:
median(durations[stds .> 0.2]), median(durations[stds .< 0.2])

In [ ]:
fig, ax = subplots()
labels = map(x -> round(x, digits = 2), I)
ax.boxplot(groups, labels = labels, showfliers = false)
ax.set(xlabel = "Coefficient de variation", ylabel = "Durée moyenne (minutes)", ylim = (-10, 310))
ax.grid(axis = "x")
save_thesis("atlas_cv_duration_boxplot", hwr = 0.5, extra_axis_params = ["xtick={$(join(1:length(labels), ","))}", "xticklabels={$(join(labels, ", "))}"])

In [ ]:
safe(f) = try f(); catch NaN; end

In [ ]:
plot(collect(0:Δ:1), map(median, groups))
plot(collect(0:Δ:1), map(x -> percentile(x, 25), groups))
plot(collect(0:Δ:1), map(x -> percentile(x, 75), groups))
# xlim(-2, 200)

In [ ]:
# using Seaborn
# fig, ax = subplots()
# kdeplot(log.(durations), log.(stds), ax = ax, shade=true)
# # ax.set_xscale("log")
# # ax.set_yscale("log")